## 0. Import libs

In [1]:
import sys
sys.path.append("src")

In [2]:
import pandas as pd

from data.examples_test import test_examples
from configs.examples_for_prompt import examples
from configs.prompts import (
    get_user_prompt_examples,
    get_system_prompt_validation
    )

from src.assessor import validate_model, make_full_assessment_few_shot, make_full_assessment_two_steps
from src.parser import validation_dataset_parser, clean_validation_dataset
from src.helpers import call_openai_api, assessments_to_df, criterions_to_df, save_dataset

pd.set_option("display.max_colwidth", None)

## 1. Reverse engineer assessor

In [4]:
# Import test example
example = test_examples[0]
question = example['question']
answer = example['answers'][0]

print(f'Selected question: \n{question}')
print(f'\nSelected answer: \n{answer}')

Selected question: 
Imagine you are a product manager at a subscription-based streaming service looking to reduce customer churn. Describe how churn-prediction insights can inform actionable pricing or content-delivery strategies for this service.

Selected answer: 
Churn prediction is a model that estimates which users are likely to cancel. It helps companies understand user behavior.


### Approach 1: Few shot classification

In [5]:
assessment = make_full_assessment_few_shot(question, answer, examples)

In [6]:
assessments_to_df([assessment], question, answer)

,question,answer,criterions,final_thought,score
0,Imagine you are a product manager at a subscription-based streaming service looking to reduce customer churn. Describe how churn-prediction insights can inform actionable pricing or content-delivery strategies for this service.,Churn prediction is a model that estimates which users are likely to cancel. It helps companies understand user behavior.,"[{'text': 'Explains what churn prediction is', 'match': 'MatchTag.yes', 'explanation': 'The answer states, ""Churn prediction is a model that estimates which users are likely to cancel.""'}, {'text': 'Connects churn prediction to understanding user behavior', 'match': 'MatchTag.yes', 'explanation': 'The answer mentions, ""It helps companies understand user behavior.""'}, {'text': 'Describes how churn prediction can inform pricing strategies', 'match': 'MatchTag.no', 'explanation': 'The answer does not mention pricing strategies.'}, {'text': 'Describes how churn prediction can inform content-delivery strategies', 'match': 'MatchTag.no', 'explanation': 'The answer does not mention content-delivery strategies.'}, {'text': 'Provides actionable strategies based on churn prediction insights', 'match': 'MatchTag.no', 'explanation': 'The answer does not provide any actionable strategies.'}]","The student answer provides a basic understanding of what churn prediction is and its role in understanding user behavior. However, it lacks any connection to actionable pricing or content-delivery strategies, which are crucial to fully addressing the question.",0.25


In [6]:
criterions_to_df([assessment])

,answer_id,criterion_text,match,explanation
0,0,Explains what churn prediction is,Yes,"The answer states, ""Churn prediction is a model that estimates which users are likely to cancel."""
1,0,Connects churn prediction to understanding user behavior,Yes,"The answer mentions, ""It helps companies understand user behavior."""
2,0,Describes how churn prediction can inform pricing strategies,No,The answer does not mention pricing strategies.
3,0,Describes how churn prediction can inform content-delivery strategies,No,The answer does not mention content-delivery strategies.
4,0,Provides actionable strategies based on churn prediction insights,No,The answer does not provide any actionable strategies.


### Approach 2: Criteria generation => Assessment

In [7]:
assessment = make_full_assessment_two_steps(question, answer, examples)

In [8]:
assessments_to_df([assessment], question, answer)

,question,answer,criterions,final_thought,score
0,Imagine you are a product manager at a subscription-based streaming service looking to reduce customer churn. Describe how churn-prediction insights can inform actionable pricing or content-delivery strategies for this service.,Churn prediction is a model that estimates which users are likely to cancel. It helps companies understand user behavior.,"[{'text': 'Explain how churn-prediction insights can identify at-risk customers and inform targeted pricing strategies to retain them.', 'match': 'MatchTag.no', 'explanation': 'The student answer mentions that churn prediction estimates which users are likely to cancel, but it does not explain how this can inform targeted pricing strategies to retain them.'}, {'text': 'Describe how churn-prediction insights can guide the development of personalized content-delivery strategies to enhance customer engagement and reduce churn.', 'match': 'MatchTag.no', 'explanation': 'The student answer does not describe how churn-prediction insights can guide the development of personalized content-delivery strategies.'}, {'text': 'Discuss how churn-prediction insights can help optimize resource allocation for retention efforts, ensuring cost-effectiveness and maximizing impact.', 'match': 'MatchTag.no', 'explanation': 'The student answer does not discuss how churn-prediction insights can help optimize resource allocation for retention efforts.'}, {'text': 'Provide examples of specific pricing or content-delivery strategies that can be implemented based on churn-prediction insights.', 'match': 'MatchTag.no', 'explanation': 'The student answer does not provide examples of specific pricing or content-delivery strategies based on churn-prediction insights.'}]",The student answer provides a basic definition of churn prediction but fails to address any of the specific criteria related to actionable strategies for pricing or content delivery based on churn-prediction insights.,0.0


In [17]:
criterions_to_df([assessment])

,answer_id,criterion_text,match,explanation
0,0,Explain how churn-prediction insights can identify at-risk customers and inform targeted pricing strategies to retain them.,No,The student answer mentions that churn prediction estimates which users are likely to cancel but does not explain how this can inform targeted pricing strategies to retain them.
1,0,Describe how churn-prediction insights can guide the development of personalized content-delivery strategies to enhance customer engagement and reduce churn.,No,The student answer does not describe how churn-prediction insights can guide the development of personalized content-delivery strategies.
2,0,Discuss the role of churn-prediction insights in optimizing resource allocation for pricing and content strategies to maximize retention efforts.,No,The student answer does not discuss the role of churn-prediction insights in optimizing resource allocation for pricing and content strategies.
3,0,Provide examples of specific pricing or content-delivery strategies that can be implemented based on churn-prediction insights.,No,The student answer does not provide examples of specific pricing or content-delivery strategies based on churn-prediction insights.


## 2. Validation dataset creation for Approach 2

In [ ]:
# (1) Create
system_prompt_validation = get_system_prompt_validation(examples_count=10)
user_prompt_validation = get_user_prompt_examples(examples)
# validation_dataset = call_openai_api(user_prompt_validation, system_prompt_validation)

# (2) Clean
dataset_cleaned = clean_validation_dataset(validation_dataset)

# (3) Save
file_path = "./examples/validation_dataset.json"
save_dataset(file_path, dataset_cleaned)

## 3. Running models against the validation dataset

In [2]:
# Read dataset
filepath = "./examples/validation_dataset.json"
validation_dataset = validation_dataset_parser(filepath)

In [3]:
criteria_df_exploded, validation_df = validate_model(
    validation_dataset=validation_dataset, 
    number_of_questions_to_process = None,
    examples_for_prompt = examples)

In [ ]:
criteria_df_exploded

,question_id,question_text,criteria
0,1,Imagine you are a data scientist at a mid-sized e-commerce company aiming to boost customer retention. Describe how LTV insights can help formulate actionable customer retention strategies for this company.,Explain the concept of LTV and its relevance to customer retention.
1,1,Imagine you are a data scientist at a mid-sized e-commerce company aiming to boost customer retention. Describe how LTV insights can help formulate actionable customer retention strategies for this company.,Describe how LTV can be used to identify high-value customers and prioritize retention efforts.
2,1,Imagine you are a data scientist at a mid-sized e-commerce company aiming to boost customer retention. Describe how LTV insights can help formulate actionable customer retention strategies for this company.,"Provide specific examples of actionable strategies that can be derived from LTV insights, such as personalized loyalty programs or targeted promotions."
3,1,Imagine you are a data scientist at a mid-sized e-commerce company aiming to boost customer retention. Describe how LTV insights can help formulate actionable customer retention strategies for this company.,Discuss how LTV insights can be used to segment customers and tailor retention strategies for different customer groups.
4,1,Imagine you are a data scientist at a mid-sized e-commerce company aiming to boost customer retention. Describe how LTV insights can help formulate actionable customer retention strategies for this company.,Explain how using LTV insights can lead to cost-effective retention efforts and maximize long-term profitability.
5,2,Explain the significance of photosynthesis in the ecosystem.,Defines photosynthesis and its basic process.
6,2,Explain the significance of photosynthesis in the ecosystem.,Explains the role of photosynthesis in producing oxygen.
7,2,Explain the significance of photosynthesis in the ecosystem.,"Describes how photosynthesis contributes to the food chain by providing energy to plants and, subsequently, to herbivores and higher trophic levels."
8,2,Explain the significance of photosynthesis in the ecosystem.,Discusses the impact of photosynthesis on carbon dioxide levels and its role in regulating the Earth's climate.
9,2,Explain the significance of photosynthesis in the ecosystem.,Highlights the overall importance of photosynthesis in maintaining ecosystem balance and supporting biodiversity.


In [5]:
validation_df

,question_id,answer_id,answer_text,score_real,score_assessed,criteria match,score_explanation,score_match
0,1,1,LTV is a financial metric that tells us how much a customer is worth. It is important for businesses to know this number.,0.00,0.00,"[No, No, No, No, No]","The student answer provides a very basic definition of LTV but fails to address any of the specific criteria related to customer retention strategies, actionable insights, or profitability.",True
1,1,2,"LTV helps businesses understand the value of their customers. A higher LTV is better because it means customers spend more, which helps retention.",0.25,0.25,"[Yes, No, No, No, No]","The student answer provides a basic explanation of LTV and its relevance to customer retention but lacks depth and detail in addressing the other criteria. It does not describe how LTV can be used to identify high-value customers, provide specific actionable strategies, discuss customer segmentation, or explain cost-effective retention efforts.",True
2,1,3,"LTV helps identify which customers are most valuable to a business. If we focus on customers with higher LTV, we can try to keep them engaged for longer, improving retention.",0.50,0.25,"[Yes, Yes, No, No, No]","The student answer addresses the basic concept of LTV and its relevance to identifying high-value customers for retention. However, it lacks specific examples of actionable strategies, discussion on customer segmentation, and explanation of cost-effectiveness and profitability related to LTV insights.",False
3,1,4,"LTV helps identify high-value customers so we can focus retention efforts on them. For example, we can offer exclusive discounts or rewards to customers with high LTV to encourage loyalty.",0.75,0.25,"[No, Yes, Yes, No, No]","The student answer addresses the identification of high-value customers and provides an example of an actionable strategy using LTV insights. However, it lacks an explanation of the concept of LTV, its relevance to customer retention, customer segmentation, and the cost-effectiveness of using LTV insights.",False
4,1,5,"LTV insights help segment customers based on their long-term value, allowing us to design tailored retention strategies. For high-LTV customers, we can invest in personalized loyalty programs and VIP perks to maintain engagement. For lower-LTV customers, we might adjust marketing spend or offer targeted promotions to increase their value. This ensures that retention efforts are cost-effective and focused on maximizing long-term profitability.",1.00,0.75,"[No, Yes, Yes, Yes, Yes]","The student answer addresses most of the criteria, providing examples of actionable strategies and discussing segmentation and cost-effectiveness. However, it lacks an explicit explanation of the concept of LTV and its relevance to customer retention.",False
5,2,1,Photosynthesis is a process used by plants to convert sunlight into energy.,0.00,0.25,"[Yes, No, No, No, No]","The student answer provides a basic definition of photosynthesis but lacks details on its broader significance in the ecosystem, such as oxygen production, food chain contribution, carbon dioxide regulation, and ecosystem balance.",False
6,2,2,Photosynthesis is important because it produces oxygen.,0.25,0.25,"[No, Yes, No, No, No]",The student answer only addresses the role of photosynthesis in producing oxygen. Other aspects of the significance of photosynthesis in the ecosystem are not covered.,True
7,2,3,Photosynthesis is crucial as it provides oxygen and is the base of the food chain.,0.50,0.50,"[No, Yes, Yes, No, No]","The student answer addresses the role of photosynthesis in producing oxygen and its contribution to the food chain. However, it lacks details on the basic process of photosynthesis, its impact on carbon dioxide levels, and its importance in maintaining ecosystem balance and supporting biodiversity.",True
8,2,4,Photosynthesis is significant because it produces oxygen and organic compounds that are essential for t

In [7]:
match_percent = validation_df['score_match'].mean()
print("Percentage of matched scores: ", match_percent)

Percentage of matched scores:  0.28


In [8]:
# Save
validation_df.to_csv("./results/validation_results.csv", index=False)
criteria_df_exploded.to_csv("./results/assessment_criteria.csv", index=False)